<a href="https://colab.research.google.com/github/nikesh11xx/eda-/blob/main/movie_recomander_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#  loading dataset
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credit = pd.read_csv('/content/tmdb_5000_credits.csv')

#Basic information from the credit and movie dataset


In [ ]:
movies.head(1)

In [ ]:
movies.shape

In [ ]:
credit.shape

In [ ]:
credit.head(1)

In [ ]:
credit['cast'].values
#

# processing data

In [ ]:
# joing both data set
movies = movies.merge(credit,on = 'title')

In [ ]:
movies.shape

In [ ]:
movies.info()

In [ ]:
# creating a list of columns
# genres
# id
# keywords
# title
# overview
# cast
# crew

In [ ]:
# extracting only a required columns

movies = movies[['genres','id','keywords','title','overview','cast','crew']]

cheking null and duplicates value

- 3 values are null in overview columns
- there is zero duplicate values in dataset

In [ ]:
# cheking null/missing values
missing_values = movies.isnull().sum()
duplicate = movies.duplicated().sum()
print(missing_values)
print('')
print(f'There are {duplicate} duplicate values in dataset')

In [ ]:
# removing all null value from dataset
movies.dropna(inplace=True)

In [ ]:
# re-checking missing values
movies.isnull().sum()

In [ ]:
movies.iloc[0].genres
# its a list of dict
# values are inside a dict and dict are inside a list and list is inside a string


In [ ]:
# importing a module ast to remove string
import ast
# creating a helper function

def convert(obj):
  l = []
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
# creating a helper function to extract actors name

def cast_name(obj):
  l = []
  count = 1
  for i in ast.literal_eval(obj):
    if count != 5:
      l.append(i['name'])
      count +=1
    else:
      break
  return l

In [ ]:
movies['cast'] = movies['cast'].apply(cast_name)

In [ ]:
# creating a helper function to extract director name

def director_name(obj):
  l = []
  for i in ast.literal_eval(obj):
    if i['job']=='Director':

      l.append(i['name'])
      break
  return l

In [ ]:
movies['crew'] = movies['crew'].apply(director_name)

In [ ]:
# converting overview columns to list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
# removing space between the words in genres,keywords,crew,cast columns
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ','')for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(' ','')for i  in x])

In [ ]:
movies.head()

In [ ]:
# creatinng a new columns name tag  by merging cast,crew,keywords,overview
movies['tags'] = movies['overview'] + movies['cast'] + movies['crew'] + movies['keywords'] + movies['genres']

In [ ]:
movies['tags']

In [ ]:
movies.head(1)

In [ ]:
# creating new dataframe
new_df = movies[['id','title','tags']]


In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:' '.join(x))

In [ ]:
new_df

In [ ]:
# converting the values of  tags columns into lower case

new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
cv.get_feature_names_out()

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [ ]:
def stem(text):
  y =[]
  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)


In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_dist = cosine_similarity(vectors)

In [ ]:
# sorting similiarity value without loosing index
sorted(list(enumerate(similarity_dist[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
def recomend(movies):
  movie_index = new_df[new_df['title']==movies].index[0]
  distances = similarity_dist[movie_index]
  movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]].title)




In [ ]:
recomend('The Avengers')

In [ ]:
import pickle


In [ ]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))